In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings 
import calendar

In [ ]:
import findspark

In [ ]:
findspark.init('/home/datasvm/spark-2.4.4-bin-hadoop2.7')

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
df = spark.read.json('ReviewSample.json')

In [ ]:
#reading mulitple json file 
reviewDataset=[]
with open(df[0]) as data_file:
    data=data_file.read()
    for i in data.split('\n'):
        reviewDataset.append(i)
#making a list of tuples from json file         
reviewDataframe=[]
for x in reviewDataset:
    try:
        jd=json.loads(x)
        reviewDataframe.append((jd['reviewerID'],jd['asin'],jd['reviewerName'],jd['helpful']
[0],jd['helpful'][1],jd['reviewText'],jd['overall'],jd['summary'],
jd['unixReviewTime'],jd['reviewTime']))    
    except:
        pass
#creating a dataframe using a list of tuples   
dataset=pd.DataFrame(reviewDataframe, columns=['Reviewer_ID','Asin','Reviewer_Name','helpful_UpVote','Total_Votes',
                                              'Review_Text','Rating','Summary','Unix_Review_Time','Review_Time'])    
                                        

In [ ]:
# convering the data type of Review_Time into DataFrame 'dataset to datetime format using pandas

In [ ]:
dataset['Review_Time'] = pd.to_datetime(dataset['Review_Time'])

In [ ]:
#creating year and month column from Review_time data

In [ ]:
dataset['Month'] = dataset['Review_Time'].dt.month

In [ ]:
dataset['Year'] = dataset['Review_Time'].dt.year

In [ ]:
#Grouping by year and taking the  count of reviews for each year

In [ ]:
Yearly = dataset.groupby(['Year'])['Reviewer_ID'].count().reset_index()

In [ ]:
Yearly = Yearly.rename(columns={'Reviewer_ID' : 'Number_Of_Reviews'})

In [ ]:
Yearly = Yearly.sort_values('Number_Of_Reviews', ascending=False)

In [ ]:
Yearly.head(10)

In [ ]:
#Line plot for number of reviews over the years 

In [ ]:
Yearly.plot(x="Year",y="Number_Of_Reviews",kind="line",title="NUMBER OF REVIEWS OVER THE YEARS")
plt.show()

 Number of reviews by Month(1996-2014)

In [ ]:
#Grouping on Month and getting the count 

In [ ]:
Monthly = dataset.groupby(['Month']) ['Reviewer_ID'].count().reset_index()

In [ ]:
#Replacing digits of 'Month' column in 'Monthly' dataframe words using 'Calender' library

In [ ]:
Monthly['Month'] = Monthly['Month'].apply(lambda x: calendar.month_name[x])
Monthly=Monthly.rename(columns={'Reviewer_ID':'Number_of_Reviews'})

In [ ]:
Monthly.head(12)

Bar chart plot for number of reviews over the month 

In [ ]:
Monthly.plot(x="Month",y="Number_of_Reviews",kind="bar",title="NUMBER OF REVIEWS OVER THE MONTH")
plt.show()

 
 Distribution Of Length of Reviews On DataSet(Amazon
 

In [ ]:
Review_Length = dataset[['Reviewer_ID', 'Reviewer_Name', 'Review_Text']]

In [ ]:
#word count 
Review_Length['Word_Length'] = Review_Length['Review_Text'].apply(lambda x: len(x.split()))

In [ ]:
#character count 
Review_Length['Character_Length']=Review_Length['Review_Text'].apply(lambda x: len(x))

In [ ]:
Char_Review_Length = Review_Length.groupby(pd.cut(Review_Length.Character_Length, np.arange(0, 1501, 100))).count()

In [ ]:
Char_Review_Length=Char_Review_Length.rename(columns={'Character_Length':'Count'})
result_Char_Review_Length=Char_Review_Length.reset_index()

In [ ]:
Word_Review_Length=Review_Length.groupby(pd.cut(Review_Length.Word_Length,np.arange(0, 801, 100))).count()
Word_Review_Length=Word_Review_Length.rename(columns={'Word_Length':'Count'})
result_Word_Review_Length=Word_Review_Length.reset_index()

In [ ]:
result_Char_Review_Length[["Character_Length","Count"]].head(10)

 Bar Graph shows distribution of length of text_reviews characters 

In [ ]:
result_Char_Review_Length.plot(x="Character_Length",y="Count",kind="bar",title="Distribution of Character Length")
plt.show()

 Word  Count Length

In [ ]:
result_Word_Review_Length[["Word_Length","Count"]].head(8)

 Bar Graph shows distribution of length of text_reviews in words

In [ ]:
result_Word_Review_Length.plot(x="Word_Length",y="Count",kind="bar",title="Distribution of Word Length")
plt.show()

Groupping on 'Review_ID' and getting the count of reviews

Distribution of reviews for a specific customer based on overall rating(reviewerID)

Taking review which is posted by [reviewerID= A1RRMZKOMZ2M7J]

In [ ]:
PNN = dataset[dataset.Reviewer_ID == 'A1RRMZKOMZ2M7J']

Function to give status based on overallRating 

In [ ]:
def ReviewCategory(score):
    if score >= 4:
        return 'pos'
    elif(score <= 2) & (score > 0):
        return 'neg'
    else:
        return 'neu'

calling function RevieCategory() for each row of Dataframe

In [ ]:
PNN['Category'] = PNN['Rating'].apply(lambda x: ReviewCategory(x))

In [ ]:
CategoryCount=PNN.groupby('Category')['Reviewer_ID'].count().reset_index()

In [ ]:
CategoryCount=CategoryCount.rename(columns={'Reviewer_ID':'Count'})

In [ ]:
CategoryCount

In [ ]:
CategoryCount.plot(x =" Category ",y = "Count",kind="pie",title="Category V/S No. of Reviews of a customer X")
plt.show()